In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from huggingface_hub import login
login(token=os.environ['HUGGINGFACEHUB_API_TOKEN'])

In [3]:
from datasets import load_dataset

In [4]:
data = load_dataset('namfam/gsm8k-vietnamese')

README.md:   0%|          | 0.00/574 [00:00<?, ?B/s]

c:\Users\NamFam\anaconda3\envs\env-01\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NamFam\.cache\huggingface\hub\datasets--namfam--gsm8k-vietnamese. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/2.16M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/441k [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/250k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6725 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/748 [00:00<?, ? examples/s]

In [5]:
import json

# Assuming `data` is a DatasetDict
dataset_split = data["train"]  # or "test", "validation", etc.

In [6]:
# Convert to JSON (list of dictionaries)
json_data = dataset_split.to_pandas()

In [7]:
json_data

,index,answer,problem,solution
0,4691,570,Jerry được thuê để làm việc trên một ngôi nhà ...,Solution:\nReason:\nJerry mất 24 giờ để sửa cá...
1,4614,150,Yasmin đã gửi một chiếc séc sinh nhật trị giá ...,Solution:\nReason:\nSố tiền 50 đô la trên chiế...
2,7081,19,Một giáo viên đã mua một số dụng cụ học tập vớ...,Solution:\nReason:\nNgân sách tổng của giáo vi...
3,5434,274,Kylie và Kayla hái táo cùng nhau và mang về nh...,Solution:\nReason:\nKylie đã hái được 66 quả t...
4,2504,70,Elsie có một số lượng khăn ướt cụ thể trong mộ...,"Solution:\nReason:\nSáng hôm đó, Elsie có w mi..."
...,...,...,...,...
6720,4306,23,Có 920 con nai trong một công viên. 10% số nai...,Solution:\nReason:\nĐầu tiên tìm số lượng nai ...
6721,6983,1260,Một chiếc vòng quay có thể chứa được 70 người ...,Solution:\nReason:\nCông Viên Vòng Quay sẽ mở ...
6722,2726,8,Jeremy thích chơi Scrabble. Anh ấy từng đánh m...,"Solution:\nReason:\nSau khi từ ""score"" được nh..."
6723,49,48,Trong chuyến đi dã ngoại đến bờ biển của trườn...,Solution:\nReason:\nBen đã sưu tầm được 12 vỏ ...


In [8]:
import json
import os
from datasets import load_dataset, Dataset, DatasetDict

def save_hf_dataset_to_json(data_id: str, save_dir: str = None):
    """
    Load a Hugging Face dataset and save each split as a JSON file.

    Args:
        data_id (str): Path to the Hugging Face dataset (or dataset ID).
        save_dir (str, optional): Directory to save JSON files. Defaults to the second part of `data_id` after '/'.
    """
    # If save_dir is not specified, extract dataset name from data_id
    if save_dir is None:
        save_dir = data_id.split("/")[-1]

    # Load the dataset
    dataset = load_dataset(data_id)

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Save each split separately
    if isinstance(dataset, DatasetDict):
        for split, ds in dataset.items():
            output_file = os.path.join(save_dir, f"{split}.json")
            data = ds.to_list()
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            print(f"Saved {split} split to {output_file}")
    elif isinstance(dataset, Dataset):
        output_file = os.path.join(save_dir, "dataset.json")
        data = dataset.to_list()
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Saved dataset to {output_file}")
    else:
        raise ValueError("Invalid dataset format. Must be a Hugging Face Dataset or DatasetDict.")

# Example usage:
# save_hf_dataset_to_json("squad")  # Saves in "squad/" directory
# save_hf_dataset_to_json("cnn_dailymail", "news_data")  # Saves in "news_data/" directory


In [9]:
save_hf_dataset_to_json('namfam/gsm8k-vietnamese', 'E:/projects/SolveItEasy/data/GSM8K/gsm8k_vietnamese')

Saved train split to E:/projects/SolveItEasy/data/GSM8K/gsm8k_vietnamese\train.json
Saved test split to E:/projects/SolveItEasy/data/GSM8K/gsm8k_vietnamese\test.json
Saved val split to E:/projects/SolveItEasy/data/GSM8K/gsm8k_vietnamese\val.json


In [31]:
from datasets import load_dataset
import os

def load_hf_dataset_from_json(save_dir: str):
    """
    Load a Hugging Face dataset from JSON files.

    Args:
        save_dir (str): Directory containing JSON files.

    Returns:
        DatasetDict or Dataset: Loaded dataset.
    """
    json_files = [f for f in os.listdir(save_dir) if f.endswith(".json")]

    if len(json_files) == 1:
        # If only one dataset file exists, load it as a single Dataset
        dataset = load_dataset("json", data_files=os.path.join(save_dir, json_files[0]))
    else:
        # If multiple JSON files exist (splits), load them into a DatasetDict
        data_files = {file.replace(".json", ""): os.path.join(save_dir, file) for file in json_files}
        dataset = load_dataset("json", data_files=data_files)
    
    return dataset

# Example usage:
# dataset = load_hf_dataset_from_json("output_dir")
# print(dataset)


In [32]:
dataset = load_hf_dataset_from_json("E:/projects/SolveItEasy/data/GSM8K/gsm8k_vietnamese")

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

In [195]:
from datasets import load_dataset

In [196]:
gsm8k_en = load_dataset('openai/gsm8k', 'main')

In [197]:
gsm8k_en['train'] = gsm8k_en['train'].add_column("index", list(range(len(gsm8k_en['train']))))
gsm8k_en['test'] = gsm8k_en['test'].add_column("index", list(range(len(gsm8k_en['test']))))

In [201]:
gsm8k_en['train'][100]

{'question': 'A craft store makes a third of its sales in the fabric section, a quarter of its sales in the jewelry section, and the rest in the stationery section. They made 36 sales today. How many sales were in the stationery section?',
 'answer': 'The craft store made 36 / 3 = <<36/3=12>>12 sales in the fabric section.\nIt made 36 / 4 = <<36/4=9>>9 sales in the jewelry section.\nThus, there were 36 - 12 - 9 = <<36-12-9=15>>15 sales in the stationery section.\n#### 15',
 'index': 100}

In [199]:
print(gsm8k_en['train'][100]['answer'].split('####')[0].strip())

The craft store made 36 / 3 = <<36/3=12>>12 sales in the fabric section.
It made 36 / 4 = <<36/4=9>>9 sales in the jewelry section.
Thus, there were 36 - 12 - 9 = <<36-12-9=15>>15 sales in the stationery section.


In [200]:
print(gsm8k_en['train'][100]['answer'].split('####')[1].strip())

15


In [202]:
gsm8k_en['train'].column_names

['question', 'answer', 'index']

In [203]:
def transform(example):
    explanation = example['answer'].split('####')[0].strip()
    answer = example['answer'].split('####')[1].strip()
    return {
        "index": example['index'],
        "problem": example["question"],
        "solution": f"Explanation:\n{explanation}\nAnswer:\n{answer}",
        "answer": answer
    }

# Apply transformation
new_gsm8k_en = gsm8k_en.map(transform, remove_columns=gsm8k_en['train'].column_names)

In [204]:
new_gsm8k_en

DatasetDict({
    train: Dataset({
        features: ['answer', 'index', 'problem', 'solution'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['answer', 'index', 'problem', 'solution'],
        num_rows: 1319
    })
})

In [209]:
new_gsm8k_en['train'][10]

{'answer': '121',
 'index': 10,
 'problem': 'A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?',
 'solution': 'Explanation:\nLet S be the number of people on the first hundred years’ ship.\nThe second hundred years’ ship had twice as many as the first, so it had 2S people.\nThe third hundred years’ ship had twice as many as the second, so it had 2 * 2S = <<2*2=4>>4S people.\nAll the ships had S + 2S + 4S = 7S = 847 people.\nThus, the ship that the monster ate in the first hundred years had S = 847 / 7 = <<847/7=121>>121 people on it.\nAnswer:\n121'}

In [210]:
gsm8k_en = new_gsm8k_en
# Splitting the dataset
val_ratio = 0.1
seed=202502
# train_test_split = hf_dataset.train_test_split(test_size=test_ratio, seed=seed)
train_val_split = gsm8k_en["train"].shuffle(seed=seed).train_test_split(test_size=val_ratio, seed=seed)  # 0.25 x 0.8 = 0.2

# Create DatasetDict
gsm8k_en = DatasetDict({
    "train": train_val_split["train"],
    "val": train_val_split["test"],
    "test": gsm8k_en["test"]
})

In [211]:
gsm8k_en

DatasetDict({
    train: Dataset({
        features: ['answer', 'index', 'problem', 'solution'],
        num_rows: 6725
    })
    val: Dataset({
        features: ['answer', 'index', 'problem', 'solution'],
        num_rows: 748
    })
    test: Dataset({
        features: ['answer', 'index', 'problem', 'solution'],
        num_rows: 1319
    })
})

In [213]:
gsm8k_en['val'][2]

{'answer': '70',
 'index': 6163,
 'problem': "The sum of the three angles of a triangle equals 250. The left angle is twice the right angle's value, and the right angle is 60 degrees. Find the value of the top angle?",
 'solution': 'Explanation:\nSince the left angle is twice the value of the right angle, the left angle is 60*2 = <<60*2=120>>120 degrees.\nThe total value of the left and right angles is 120+60 = <<120+60=180>>180 degrees.\nSince the sum of the three angles is equal to 250, the other angle has a value of 250-180 = <<250-180=70>>70 degrees.\nAnswer:\n70'}

In [216]:
gsm8k_en.push_to_hub('namfam/gsm8k-english')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/namfam/gsm8k-english/commit/3363fbd2541ccd2feda9390d71ee5f3da0714462', commit_message='Upload dataset', commit_description='', oid='3363fbd2541ccd2feda9390d71ee5f3da0714462', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/namfam/gsm8k-english', endpoint='https://huggingface.co', repo_type='dataset', repo_id='namfam/gsm8k-english'), pr_revision=None, pr_num=None)

In [217]:
save_hf_dataset_to_json('namfam/gsm8k-english', 'E:/projects/SolveItEasy/data/GSM8K/gsm8k_english')

README.md:   0%|          | 0.00/599 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/243k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/430k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6725 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/748 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Saved train split to E:/projects/SolveItEasy/data/GSM8K/gsm8k_english\train.json
Saved val split to E:/projects/SolveItEasy/data/GSM8K/gsm8k_english\val.json
Saved test split to E:/projects/SolveItEasy/data/GSM8K/gsm8k_english\test.json


In [218]:
gmsk_vi = load_dataset('namfam/gsm8k-vietnamese')

In [219]:
gmsk_vi

DatasetDict({
    train: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 6725
    })
    test: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 1319
    })
    val: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 748
    })
})

In [220]:
gmsk_vi['val'][2]

{'index': 6163,
 'answer': '70',
 'problem': 'Tổng ba góc của một tam giác bằng 250 độ. Góc bên trái gấp đôi giá trị của góc bên phải và góc bên phải là 60 độ. Tìm giá trị của góc đỉnh?',
 'solution': 'Explanation:\nVì góc bên trái gấp đôi góc bên phải, nên góc bên trái có giá trị là 60*2 = 120 độ.\nTổng giá trị của góc bên trái và góc bên phải là 120+60 = 180 độ.\nVì tổng ba góc bằng 250 độ, nên góc còn lại có giá trị là 250-180 = 70 độ.\nAnswer:\n70'}

In [221]:
gsm8k_en['val'][2]

{'answer': '70',
 'index': 6163,
 'problem': "The sum of the three angles of a triangle equals 250. The left angle is twice the right angle's value, and the right angle is 60 degrees. Find the value of the top angle?",
 'solution': 'Explanation:\nSince the left angle is twice the value of the right angle, the left angle is 60*2 = <<60*2=120>>120 degrees.\nThe total value of the left and right angles is 120+60 = <<120+60=180>>180 degrees.\nSince the sum of the three angles is equal to 250, the other angle has a value of 250-180 = <<250-180=70>>70 degrees.\nAnswer:\n70'}

In [222]:
for item in gmsk_vi['train']:
    if item['index'] == 99:
        print(item)

{'index': 99, 'answer': '3', 'problem': 'Gail có hai bể cá. Bể thứ nhất gấp đôi kích thước của bể thứ hai. Có 48 gallon nước trong bể thứ nhất. Cô ấy tuân theo quy tắc một gallon nước cho mỗi inch cá. Nếu cô ấy giữ cá 2 inch trong bể thứ hai và cá 3 inch trong bể thứ nhất, cô ấy sẽ có bao nhiêu cá hơn trong bể thứ nhất so với bể thứ hai nếu một con cá trong bể thứ nhất ăn mất một con cá khác?', 'solution': 'Explanation:\nBể thứ hai có dung tích 24 gallon vì 48 chia 2 bằng 24.\nTheo quy tắc của Gail, cô ấy nuôi 12 con cá dài 2 inch trong bể thứ hai vì 24 chia 2 bằng 12.\nCô ấy nuôi 16 con cá trong bể thứ nhất vì 48 chia 3 bằng 16.\nNếu một con cá trong bể thứ nhất ăn mất một con cá khác, cô ấy sẽ có 15 con cá trong bể thứ nhất.\nDo đó, Gail sẽ có thêm 3 con cá trong bể thứ nhất.\nAnswer:\n3'}


In [223]:
for item in ds:
    if item['index'] == 99:
        print(item)

{'question': 'Gail has two fish tanks. The first tank is twice the size of the second tank. There are 48 gallons of water in the first tank. She follows the rule of one gallon of water per inch of fish. If she keeps two-inch fish in the second tank and three-inch fish in the first tank, how many more fish would Gail have in the first tank than the second tank if one of the first tank fish eats another?', 'answer': 'The second tank is 48 / 2 = <<48/2=24>>24 gallons.\nFollowing her rule, Gail keeps 24 / 2 = <<24/2=12>>12 two-inch fish in the second tank.\nShe keeps 48 / 3 = <<48/3=16>>16 fish in the first tank.\nIf one fish in the first tank ate another, she would have 16 - 1 = <<16-1=15>>15 fish in the first tank.\nThus, Gail would have 15 - 12 = <<15-12=3>>3 more fish in the first tank.\n#### 3', 'index': 99}


# Merge datasets

In [45]:
from datasets import load_dataset

In [46]:
gsm8k_vi = load_dataset('namfam/gsm8k-vietnamese')
gsm8k_en = load_dataset('namfam/gsm8k-english')

In [47]:
gsm8k_vi

DatasetDict({
    train: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 6725
    })
    test: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 1319
    })
    val: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 748
    })
})

In [44]:
# gsm8k_vi = gsm8k_vi.select(range(10))
# gsm8k_en = gsm8k_en.select(range(10))

In [48]:
from datasets import DatasetDict, concatenate_datasets

def merge_datasets(dataset1: DatasetDict, dataset2: DatasetDict, shuffle: bool = False, seed: int = 42) -> DatasetDict:
    """
    Merge two Hugging Face datasets (DatasetDict) with the same column structure.
    
    Args:
        dataset1 (DatasetDict): First dataset (e.g., gsm8k_vi).
        dataset2 (DatasetDict): Second dataset (e.g., gsm8k_en).
        shuffle (bool): Whether to shuffle the merged dataset.
        seed (int): Random seed for shuffling.

    Returns:
        DatasetDict: Merged dataset.
    """
    merged_datasets = {}

    for split in dataset1.keys():  # Assume both have the same splits (train, test, etc.)
        if split in dataset2:
            merged_datasets[split] = concatenate_datasets([dataset1[split], dataset2[split]])
        else:
            merged_datasets[split] = dataset1[split]

    # Convert dict to DatasetDict
    merged_datasets = DatasetDict(merged_datasets)

    # Shuffle each split if shuffle=True
    if shuffle:
        merged_datasets = DatasetDict({
            split: dataset.shuffle(seed=seed) for split, dataset in merged_datasets.items()
        })

    return merged_datasets




In [49]:
# Example usage
gsm8k_vi_en = merge_datasets(gsm8k_vi, gsm8k_en, shuffle=True, seed=202502)
print(gsm8k_vi_en)

DatasetDict({
    train: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 13450
    })
    test: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 2638
    })
    val: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 1496
    })
})


In [50]:
gsm8k_vi_en['train'][1]

{'index': 2132,
 'answer': '2',
 'problem': 'The basketball team sold 50 cupcakes for $2 each and 40 cookies for $0.5 each. Then the money from their sales was used to buy two basketballs that cost $40 each. The rest of the money was used to buy 20 bottles of energy drinks. How much does a bottle of energy drink cost?',
 'solution': 'Explanation:\nThe basketball team earned 50 x $2 = $<<50*2=100>>100 from selling cupcakes.\nThey also earned 40 x 0.5 = $<<40*0.5=20>>20 from selling cookies.\nSo their total earnings was $100 + $20 = $<<100+20=120>>120.\nThe two basketballs cost $40 x 2 = $<<40*2=80>>80.\nThus, $120 - $80 = $<<120-80=40>>40 was used to buy the 20 bottles of energy drinks.\nTherefore, each bottle of energy drink costs $40/20 = $<<40/20=2>>2.\nAnswer:\n2'}

In [51]:
gsm8k_vi_en['train'][6726]

{'index': 3520,
 'answer': '8',
 'problem': 'Haley is getting ready to watch a comet fly over her house. She spends two hours shopping for a telescope, half an hour getting everything set up in the backyard, three times the setup time making snacks, and 20 minutes watching the comet. What percentage of the total time she spent on all those activities was spent watching the comet, rounded to the nearest percent?',
 'solution': "Explanation:\nFirst convert Haley's shopping time to minutes: 2 hours * 60 minutes/hour = <<2*60=120>>120 minutes\nThen convert Haley's setup time to minutes: .5 hours * 60 minutes/hour = <<.5*60=30>>30 minutes\nThen find the total time she spends making a snack: 3 * 30 minutes = <<3*30=90>>90 minutes\nThen add the time she spends on each activity to find the total time: 120 minutes + 30 minutes + 90 minutes = <<120+30+90=240>>240 minutes\nThen divide the time she spends watching the comet by the total time and multiply by 100% to express the answer as a percenta

In [52]:
gsm8k_vi_en.push_to_hub('namfam/gsm8k-vi-en')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/namfam/gsm8k-vi-en/commit/7b62e62067325f3d72802a1b7cb5870605815f74', commit_message='Upload dataset', commit_description='', oid='7b62e62067325f3d72802a1b7cb5870605815f74', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/namfam/gsm8k-vi-en', endpoint='https://huggingface.co', repo_type='dataset', repo_id='namfam/gsm8k-vi-en'), pr_revision=None, pr_num=None)

In [40]:
from datasets import DatasetDict, concatenate_datasets

def merge_datasets_sampled(
    dataset1: DatasetDict, dataset2: DatasetDict, num_samples: int, shuffle: bool = False, seed: int = 42
) -> DatasetDict:
    """
    Merge two Hugging Face datasets (DatasetDict) while keeping the same total size.
    Each dataset is randomly sampled based on `num_samples` before merging.

    Args:
        dataset1 (DatasetDict): First dataset (e.g., gsm8k_vi).
        dataset2 (DatasetDict): Second dataset (e.g., gsm8k_en).
        num_samples (int): Number of samples to take from each dataset before merging.
        shuffle (bool): Whether to shuffle the merged dataset.
        seed (int): Random seed for sampling and shuffling.

    Returns:
        DatasetDict: Merged dataset with the specified number of samples from each dataset.
    """
    merged_datasets = {}

    for split in dataset1.keys():  # Assume both datasets have the same splits (train, test, etc.)
        if split in dataset2:
            max_samples = min(len(dataset1[split]), len(dataset2[split]), num_samples)
            
            # Sample `num_samples` from each dataset
            sampled_ds1 = dataset1[split].shuffle(seed=seed).select(range(max_samples))
            sampled_ds2 = dataset2[split].shuffle(seed=seed).select(range(max_samples))

            # Merge the sampled datasets
            merged_datasets[split] = concatenate_datasets([sampled_ds1, sampled_ds2])
        else:
            merged_datasets[split] = dataset1[split]

    # Convert dict to DatasetDict
    merged_datasets = DatasetDict(merged_datasets)

    # Shuffle each split if shuffle=True
    if shuffle:
        merged_datasets = DatasetDict({
            split: dataset.shuffle(seed=seed) for split, dataset in merged_datasets.items()
        })

    return merged_datasets

# Example usage: Take 2000 samples from each dataset before merging
gsm8k_vi_en = merge_datasets_sampled(gsm8k_vi, gsm8k_en, num_samples=2000, shuffle=True, seed=202502)
print(gsm8k_vi_en)


DatasetDict({
    train: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 2638
    })
    val: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 1496
    })
})


In [41]:
# Example usage
gsm8k_vi_en = merge_datasets_sampled(gsm8k_vi, gsm8k_en, shuffle=True, num_samples=1000, seed=202502)
print(gsm8k_vi_en)


DatasetDict({
    train: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 2000
    })
    val: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 1496
    })
})


# Prepare data for Seq2Seq Model

In [1]:
from datasets import load_dataset

In [2]:
gsm8k_vi = load_dataset('namfam/gsm8k-vietnamese')

In [3]:
gsm8k_vi 

DatasetDict({
    train: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 6725
    })
    test: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 1319
    })
    val: Dataset({
        features: ['index', 'answer', 'problem', 'solution'],
        num_rows: 748
    })
})

In [8]:
from pprint import pprint

In [11]:
pprint(gsm8k_vi['train']['problem'][1])

('Yasmin đã gửi một chiếc séc sinh nhật trị giá $50 từ bà của cô ấy vào tài '
 'khoản ngân hàng của mình. Séc này có giá trị bằng một phần tư số dư mới của '
 'cô ấy sau khi tiền từ chiếc séc được cộng vào. Hỏi trước khi gửi chiếc séc, '
 'số tiền trong tài khoản của Yasmin là bao nhiêu đô la?')


In [15]:
pprint(gsm8k_vi['train']['solution'][1])

('Explanation:\n'
 'Số tiền 50 đô la trên chiếc séc đó tương đương với 1/4 số dư mới của Yasmin, '
 'vì vậy số dư mới của cô ấy là 50 * 4 = 200 đô la.\n'
 'Do đó, trước khi gửi séc vào tài khoản, Yasmin đã có 150 đô la trong tài '
 'khoản của mình.\n'
 'Answer:\n'
 '150')


# Function generate_prompt

In [405]:
import yaml

def load_yaml(file_path: str) -> dict:
    """Loads user data from a YAML file."""
    with open(file_path, "r", encoding="utf-8") as file:
        data = yaml.safe_load(file)
    return data.get("prompt", {})  # Extract the 'prompt' section


def format_examples(user_input: dict) -> str:
    """Formats the examples section if examples_list is provided, using examples_template."""
    examples_key = user_input.get("examples_key", "")
    examples_template = user_input.get("examples_template", "")
    examples_list = user_input.get("examples_list", [])

    if examples_key and examples_list:
        formatted_examples = "\n".join(
            examples_template.format(
                index=example.get("index", ""),
                problem=example.get("problem", ""),
                reason=example.get("reason", ""),
                answer=example.get("answer", "")
            ).strip()
            for example in examples_list if "problem" in example and "answer" in example
        )
        return f"{examples_key}\n{formatted_examples}" if formatted_examples else ""

    return ""


def generate_prompt(prompt_args: dict, phase: str) -> str:
    """
    Generates a structured prompt based on the provided dictionary.
    If any key or text is missing, that section is excluded.

    Sections are included only if:
    - Their corresponding key/text is present.
    - The relevant flag allows their inclusion.

    Args:
        prompt_args (dict): A dictionary containing keys and their corresponding text.
        phase (str): The phase of the process (e.g., 'train').

    Returns:
        str: The formatted prompt.
    """

    # Boolean flags to control sections
    use_only_input_text = prompt_args.get("use_only_input_text", False)
    use_examples = prompt_args.get("use_examples", False)
    use_context = prompt_args.get("use_context", False)
    use_response_format_guide = prompt_args.get("use_response_format_guide", False)

    # If `use_only_input_text` is True, return only input-related sections
    if use_only_input_text:
        return prompt_args.get('input_text', '')

    sections = [
        ("", "intro_text"),
        ("instruction_key", "instruction_text"),
        ("response_format_guide_key", "response_format_guide_text") if use_response_format_guide else ('', ''),  # Exclude if flag is False
        ("examples_key", ["examples_template", "examples_list"]) if use_examples else ('', ''),  # Exclude if flag is False
        ("context_key", "context_text") if use_context else ('', ''),  # Exclude if flag is False
        ("input_key", "input_text"),
        ("response_key", "response_text") if phase == 'train' else ("response_key", ""),
    ]

    prompt_parts = []

    for key, text in sections:
        section_content = []

        if key == "examples_key":
            section_content = format_examples(prompt_args)
            prompt_parts.append(section_content)
            continue

        if key in prompt_args and prompt_args[key]:  # If section_key exists and is not empty
            section_content.append(prompt_args[key])

        if text in prompt_args and prompt_args[text]:  # If section_text exists and is not empty
            section_content.append(prompt_args[text])

        if section_content:
            prompt_parts.append("\n".join(section_content))  # Join key and text if both exist

    prompt = "\n---\n".join(prompt_parts)

    if "end_key" in prompt_args and prompt_args["end_key"]:
        prompt += prompt_args["end_key"]

    return prompt




In [406]:
# Load user data from YAML file
yaml_file_path = "E:/projects/SolveItEasy/configs/toy_generate.yaml"  # Change this to your actual YAML file path
user_data = load_yaml(yaml_file_path)
phase = 'train'
# user_data

In [407]:
# Generate and print the final prompt
final_prompt = generate_prompt(user_data, phase)
print(final_prompt)


You are a math expert.
---
Solve the following problem carefully and accurately.
Your response must include two sections:
- Reason: Provide a step-by-step explanation of how to solve the problem.
- Answer: Write only the final numerical answer without any additional text, symbols, or units.
---
Format your response exactly as follows:
Solution:
Reason:
<Step-by-step explanation>
Answer:
<Final numerical result>
---
Here are some examples to guide you:
- Example 1
  Problem:
  Weng kiếm được $12 một giờ khi trông trẻ. Hôm qua, cô ấy chỉ trông được 50 phút. Cô ấy kiếm được bao nhiêu tiền?

  Solution:
  Reason:
  Weng kiếm được 12/60 = $0.2 mỗi phút.
Làm việc trong 50 phút, cô ấy kiếm được 0.2 x 50 = $10.
  Answer:
  10
- Example 2
  Problem:
  Weng kiếm được $12 một giờ khi trông trẻ. Hôm qua, cô ấy chỉ trông được 50 phút. Cô ấy kiếm được bao nhiêu tiền?

  Solution:
  Reason:
  Weng kiếm được 12/60 = $0.2 mỗi phút.
Làm việc trong 50 phút, cô ấy kiếm được 0.2 x 50 = $10.
  Answer:
  10
